# Dynamics of Kirchhoff Plates

$D\nabla^2\nabla^2w = q - \rho h\ddot{w}$,

with $w$ being the displacement normal to the plate, $q$ external normal stress applied along the plate, $h$ the plate thickness. Moreover

$D = \frac{Eh^3}{12(1-\nu^2)}$,

with $\rho$, $E$ and $\nu$ being the plate constitutive properties, respectively its density, Young's modulus and Poisson's ratio.

## Vibration of axisymmetric plates

For circular plates in cylindrical coordinates and axisymmetric conditions $w=w(r,t)$, the equation of motion becomes:

$D\Big(\frac{\partial^4 w}{\partial r^4}+\frac{2}{r}\frac{\partial^3 w}{\partial r^3}-\frac{1}{r^2}\frac{\partial^2 w}{\partial r^2}+\frac{1}{r^3}\frac{\partial w}{\partial r}\Big) = q(r,t) - \rho h \frac{\partial^2 w}{\partial t^2}$.

Neglecting the effect of external stress $q$ (free vibration) and applying the following separation of variables $w(r,t) = W(r)F(t)$, one gets

$F\Big(\frac{d^4 W}{dr^4}+\frac{2}{r}\frac{d^3 W}{dr^3}-\frac{1}{r^2}\frac{d^2 W}{d r^2}+\frac{1}{r^3}\frac{d W}{d r}\Big) = \frac{-\rho h}{D}W\frac{d^2F}{dt^2}$

Two functions of independent variables that are always equal must be constant, i.e.:

$$
\left\lbrace
\begin{align}
\omega^2 &= -\frac{1}{F}\frac{d^2F}{dt^2}\\
\omega^2 &= \frac{D}{\rho h}\frac{1}{W}\Big(\frac{d^4 W}{dr^4}+\frac{2}{r}\frac{d^3 W}{dr^3}-\frac{1}{r^2}\frac{d^2 W}{d r^2}+\frac{1}{r^3}\frac{d W}{d r}\Big)
\end{align}
\right.
$$

In [ ]:
import sympy as syp
from sympy.abc import r,t
F = syp.Function("F")(t)
om = syp.Symbol("\omega")
equ = syp.Eq(om**2,-1/F*syp.Derivative(F,t,2))
syp.dsolve(equ)

$\omega$ is then the pulsation

In [ ]:
W = syp.Function("W")(r)
lam = syp.Symbol("\lambda")
rhs = syp.Derivative(W,r,4)+2/r*syp.Derivative(W,r,3)-1/r**2*syp.Derivative(W,r,2)+1/r**3*syp.Derivative(W,r,1)
lhs = lam**4 * W 
equ = syp.Eq(lhs,rhs)
equ

with $\lambda^4 = \omega^2\rho h/D$.

In [ ]:
syp.dsolve(equ)

Oups! We can still verify that $W(r)=C_3J_0(\lambda r) + C_4I_0(\lambda r)$ satisfies the equation:

In [ ]:
c3, c4 = syp.symbols("C_3 C_4")
sol = c3*syp.besselj(0,lam*r)+c4*syp.besseli(0,lam*r)
expr = rhs.subs([(W,sol)])
syp.simplify(expr.doit())

The constants $C_3$ and $C_4$ are then determined by the boundary conditions. For instance, a plate clamped along its edge requires that both $W$ and its first derivative are zero at $r=R$:

$C_3J_0(\lambda R) + C_4I_0(\lambda R)=-C_3\lambda J_1(\lambda R) + C_4\lambda I_0(\lambda R)=0,$

which leads to the following equation whose roots correspond to the modal frequencies of the plate:

$J_0(\lambda R)I_1(\lambda R) + J_1(\lambda R)I_0(\lambda R) = 0$

In [ ]:
import scipy.special as sp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
r = np.linspace(0,10,100)
fig1,ax1 = plt.subplots()
ax1.plot(r,sp.jv(0,r)*sp.iv(1,r) + sp.jv(1,r)*sp.iv(0,r),"r")
ax1.plot(r,np.zeros(r.shape),"--k")
ax1.set_ylim(-5,5)
ax1.set_xlabel("$\lambda R$")
ax1.set_ylabel("$J_0(\lambda R)I_1(\lambda R) + J_1(\lambda R)I_0(\lambda R)$")

$\lambda_n R = n\pi,\; n=0,1,2,3,...$ provides a good estimate of the first roots:

In [ ]:
ax1.plot(np.pi*np.array([0,1,2,3]),np.zeros(4),'bo')
fig1

From the definition of $\lambda$, the pulsations of a clamped Kirchhoff plate can then be estimated as:

\begin{equation}
\omega_n = \lambda_n^2\sqrt{\frac{D}{\rho h}} \cong \Big(\frac{n\pi}{R}\Big)^2\sqrt{\frac{Eh^3}{12\rho h(1-\nu^2)}}
\end{equation}

The general solution of clamped plate vibration corresponds then to
\begin{equation}
w(r,t) = \sum_{n} C_3\Big(J_0(\lambda_n r) - \frac{J_0(\lambda_n R)}{I_0(\lambda_nR)} I_0(\lambda_n r)\Big) \Big(C_1e^{i\omega_nt}+C_2e^{-i\omega_nt}\Big) 
\end{equation}

The exact values of the pulsation, as well as the associated plate deformation can also be estimated numerically:

In [ ]:
n=3 #Mode number
lamR = syp.Symbol("\lambda R")
res = syp.nsolve(syp.besselj(0,lamR)*syp.besseli(1,lamR)+syp.besselj(1,lamR)*syp.besseli(0,lamR),lamR,n*np.pi)
root = float(res)
fig2,ax2 = plt.subplots()
tilder = np.linspace(0,1,100)
w = sp.jv(0,tilder*root) - sp.iv(0,tilder*root) * sp.jv(0,root)/sp.iv(0,root)
ax2.plot(tilder,w/w[0],'r')
ax2.set_xlabel('$r/R$')
ax2.set_ylabel('$W_'+str(n)+'(r)/C_3$')
ax2.set_title('$\lambda_'+str(n)+'R/\pi='+str(root/np.pi)+'$')

## Finite element approximation

The finite element method solves the plate equation of motion in its variational (or weak) form:

\begin{equation}
D\int_0^R \Big(\nabla^2w(r,t) \nabla^2\tilde{w}(r,t) rdr\Big) = \int_0^R\Big(q(r,t)\tilde{w}(r,t) rdr\Big) -\rho h \int_0^R\Big(\ddot{w}(r,t)\tilde{w}(r,t) rdr\Big),
\end{equation}

with $\tilde{w}$ being an arbitrary test function of differentiability class $C^2$ that is zero on the region of the plate with imposed-displacement boundary conditions. (I will write a separate note detailing how to obtain this variational form). 
Next the plate of interest is discretized with sampling points ($=$nodes) from which the displacement field is interpolated over space following:

$$
w(r,t) = \sum_i N_i(r)w_i(t) = \underline{N}(r)\cdot\underline{d}(t),
$$

with $\underline{d}$ a vector containing the values of the unknown field $w$ at the nodes and $\underline{N}$ the associated vector containing the interpolation functions (aka shape functions). Using the same interpolation for $\tilde{w}(x,t)= \underline{N}(r)\cdot\underline{\tilde{d}}(t)$, the variational form can be written as

\begin{equation}
D\int_0^R \Big(\nabla^2\underline{N}\underline{d} \nabla^2\underline{N}\underline{\tilde{d}} rdr\Big) = \int_0^R\Big(q(r,t)\underline{N}\underline{\tilde{d}} rdr\Big) -\rho h \int_0^R\Big(\underline{N}\underline{\ddot{d}}\underline{N}\underline{\tilde{d}} rdr\Big).
\end{equation}

The displacement vectors do not depend on $r$ and could be taken out of the integral by rewritting it as:

\begin{equation}
\underline{\tilde{d}}^{\top}D\int_0^R \Big(\nabla^2\underline{N}^\top\nabla^2\underline{N}rdr\Big)\underline{d} = \underline{\tilde{d}}^{\top}\int_0^R\Big(\underline{N}^\top q(r,t)rdr\Big) - \underline{\tilde{d}}^{\top}\rho h\int_0^R\Big(\underline{N}^\top\underline{N} rdr\Big)\underline{\ddot{d}}.
\end{equation}

As the test function is arbitrary, $\underline{\tilde{d}}^\top$ can be dropped in the equation above which takes then the form:
\begin{equation}
{\bf K}\underline{d} = \underline{f}_{\rm ext} - {\bf M}\underline{\ddot{d}},
\end{equation}
with ${\bf K}$ and ${\bf M}$ being respectively referred to as the stiffness and mass matrices. $\underline{f}_{\rm ext}$ is the vector of externally applied forces.

The strategy behind finite element modelling is to pick up interpolation functions that are piecewise polynomials taking finite values over some portion of the domain (aka the element) and being zero elsewhere. In the 1D geometry of interest, an element simply corresponds to a segment with two nodes at its edges. To guarantee the convergence of the numerical scheme, the choice of these polynomials must respect mainly two criteria:
1) The interpolation should be of class $C^{m-1}$ across the element boundaries.

2) The interpolation should be of class $C^{m}$ in the element and the polynomials should be complete up to degree $m$.

$m$ corresponds to the order of the variational form of the equation, $m=2$ for Kirchhoff plate equation. 
1) requires both $w$ and $\partial w/\partial r\equiv\theta$ to be continuous between two elements, which could be achieved by settings both $w$ and its first derivative $\theta$ as nodal unknowns (aka degrees of freedom). The displacements vector for an element reads then
\begin{equation}
\underline{d}^e \equiv \lbrace w_1, \theta_1, w_2, \theta_2\rbrace,
\end{equation}
with the subscript $1$ and $2$ referring respectively to the left and right nodes of the element.

2) can be satisfied by using Hermite shape functions. They correspond to third-order polynomials defined as the displacement field obtained through an element by setting the degree of freedom of interest to one and the three others to zero. (This might become clearer after looking at the next cell).

In [ ]:
#Computing Hermite shape functions for one element
from sympy.abc import a,b,c,d
dofs_per_node = 2
nodes_per_elem = 2
r, r1, r2 = syp.symbols('r r_1 r_2')
uns = np.array([[1,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,1]])
N = []
for u in range(0,dofs_per_node*nodes_per_elem):
    shape = syp.solve_poly_system([a*r1*r1*r1+b*r1*r1+c*r1+d-uns[0,u],a*r2*r2*r2+b*r2*r2+c*r2+d-uns[1,u],3*a*r1*r1+2*b*r1+c-uns[2,u],3*a*r2*r2+2*b*r2+c-uns[3,u]], a, b, c, d)
    func = shape[0][0]*r*r*r + shape[0][1]*r*r + shape[0][2]*r + shape[0][3]
    N.append(syp.factor(func.simplify()))

In [ ]:
#Display the shape function of interest (for an element between r=1 and r=2)
shape_no = 2
syp.plot(N[shape_no].subs([(r1,1),(r2,2)]),(r,1,2),title='$N= $'+str(N[shape_no]),ylabel=None)

Once the shape functions have been properly defined, one can compute the element stiffess matrix (for the nodal positions $r=r_1$ and $r=r_2$):
\begin{equation}
{\bf K}^e \equiv D\int_{r_1}^{r_2} \Big(\nabla^2\underline{N}^\top\nabla^2\underline{N}rdr\Big)
\end{equation}
and the element mass matrix
\begin{equation}
{\bf M}^e \equiv \rho h\int_{r_1}^{r_2} \Big(\underline{N}^\top\underline{N}rdr\Big).
\end{equation}

In [ ]:
#Taking D=1, \rho h=1 (could be readily set to dimensional values later)
k = []
m = []
for i in range(0,dofs_per_node*nodes_per_elem):
    k.append([])
    m.append([])
    for j in range(i,dofs_per_node*nodes_per_elem):
        nabla_Nt = syp.simplify(syp.diff(r*(syp.diff(N[i],r)),r)/r)
        nabla_N = syp.simplify(syp.diff(r*(syp.diff(N[j],r)),r)/r)
        k[i].append(syp.factor(syp.integrate(nabla_Nt*nabla_N*r,(r,r1,r2))))
        m[i].append(syp.factor(syp.integrate(N[i]*N[j]*r,(r,r1,r2))))
Kalg = syp.Matrix(4,4,lambda i,j: k[i][j-i] if(j>=i) else k[j][i-j])
Malg = syp.Matrix(4,4,lambda i,j: m[i][j-i] if(j>=i) else m[j][i-j])

Solving the finite element problem consists then by discretizing the plate of interest with elements

In [ ]:
plate_radius = 1.0
nb_nodes = 64
nodes = np.linspace(0.,plate_radius,nb_nodes)
nb_elements = nb_nodes-1 

and computing the global matrices ${\bf K}$ and ${\bf M}$ as an assembly of the element matrices ${\bf K}_e$ and ${\bf M}_e$ evaluated for each element. The resulting ${\bf K}$ and ${\bf M}$ are sparse matrices of size _nb_dofs_ x _nb_dofs_.

In [ ]:
#Can take some time for fine mesh
import scipy.sparse as spa
from tqdm import tqdm
K = spa.lil_matrix((nb_nodes*dofs_per_node,nb_nodes*dofs_per_node),dtype=np.float64)
M = spa.lil_matrix((nb_nodes*dofs_per_node,nb_nodes*dofs_per_node),dtype=np.float64)
for e in tqdm(range(0,nb_elements)):
    n1 = e
    n2 = e+1
    for l in range(0, dofs_per_node*nodes_per_elem):
        for c in range(0, dofs_per_node*nodes_per_elem):
            K[n1*dofs_per_node+l,n1*dofs_per_node+c] += syp.limit(syp.limit(Kalg[l,c], r1, nodes[n1]), r2, nodes[n2])
            M[n1*dofs_per_node+l,n1*dofs_per_node+c] += syp.limit(syp.limit(Malg[l,c], r1, nodes[n1]), r2, nodes[n2])

Using this finite element approach, we now have some freedom to model different types of problems and boundary conditions.

### Example: modal analysis

For instance, we can repeat numerically the modal analysis presented above. Neglecting again the effect of externally applied stress and using the same decomposition $w(r,t)=W(r)\Big(C_1e^{i\omega_nt}+C_2e^{-i\omega_nt}\Big)$, the finite element equation of motion becomes:
\begin{equation}
{\bf K}\underline{d} = \omega^2{\bf M}\underline{d},
\end{equation}
which leads to the following eigenvalue problem
\begin{equation}
{\bf M}^{-1}{\bf K}\underline{d} = \omega^2\underline{d}.
\end{equation}

First, we need to apply the boundary conditions, which implies to remove the blocked degrees of freedom and their associated rows and columns in ${\bf K}$ and ${\bf M}$.

In [ ]:
def removeRowsAndColumns(smat,indexes):
    for index in np.sort(indexes)[::-1]:
        smat.rows = np.delete(smat.rows, index)
        smat.data = np.delete(smat.data, index)
        l=0
        for list in smat.rows:
            i=0
            while i<len(list):
                if(list[i]>index):
                    list[i] -= 1
                elif(list[i]==index):
                    del list[i]
                    del smat.data[l][i]
                    i-=1
                i+=1
            l+=1
        smat._shape = (smat._shape[0]-1,smat._shape[1]-1)
    return

Axisymmetric condition always requires to block the second degree of freedom, such that $\partial w/\partial r$ is zero at the center of the plate. Moreover, to reproduce the conditions assumed previously, we need to clamp the plate at $r=R$ and hence block the two degrees of freedom of the last node.

In [ ]:
blocked_dofs = np.array([1,nb_nodes*dofs_per_node-2,nb_nodes*dofs_per_node-1])
removeRowsAndColumns(K,blocked_dofs)
removeRowsAndColumns(M,blocked_dofs)

Then we can solve the eigenvalue problem:

In [ ]:
import scipy.sparse.linalg
#More efficient format for matrix operations
K = K.tocsc()
M = M.tocsc()
invMK = spa.linalg.inv(M)@K
#Solve for the first three modes
puls, displ = spa.linalg.eigs(invMK,k=3,sigma=0,which='LM')

and display the result:

In [ ]:
n=3 #Mode number
index = np.argsort(puls)[n-1]
#Extract the displacements from the dofs (ignore the derivatives)
w_fem = displ[1::2,index].real/displ[0,index].real
w_fem = np.insert(w_fem,0,1.0)
w_fem = np.insert(w_fem,-1,0.)
fig3,ax3 = plt.subplots()
ax3.plot(nodes,w_fem,'.-g')
ax3.set_xlabel('$r/R$')
ax3.set_ylabel('$W_'+str(n)+'(r)/C_3$')
ax3.set_title('$\omega_'+str(n)+'\sqrt{\\rho h/D}/(n\\pi R)^2='+str(np.sqrt(puls[index].real)/((n*np.pi)**2))+'$')

In [ ]:
#Superposition on the plot of the theoretical solution constructed previously
ax2.plot(nodes,w_fem,'.g')
fig2